In [2]:
import gzip
import shutil
import pysam
import reportlab

In [9]:
source_path = '/Users/yijiasun77/Desktop/Intern/Rayca/normal_sample.deepvariant.vcf.gz'
destination_path = '/Users/yijiasun77/Desktop/Intern/Rayca/normal_sample.deepvariant.vcf'

with gzip.open(source_path, 'rb') as f_in:
    with open(destination_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [1]:
import pysam
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer

def parse_vcf(vcf_file):
    vcf = pysam.VariantFile(vcf_file)
    variants = []
    for rec in vcf.fetch():
        for alt in rec.alts:
            variants.append({
                'chromosome': rec.chrom,
                'position': rec.pos,
                'ref_allele': rec.ref,
                'alt_allele': alt,
                'quality': rec.qual,
                'filter': rec.filter.keys(),
                'info': rec.info
                
            })
    return variants


def classify_variant(variant):
    return 'Pathogenic' if variant['quality'] > 30 else 'Likely Pathogenic'


def generate_pdf_report(variants, pdf_filename):
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
    story = []
    styles = getSampleStyleSheet()
    
    for variant in variants:   

        story.append(Paragraph('<b>Patient ID:</b> {}'.format(patient_info['Patient ID']), styles['Normal']))
        story.append(Paragraph('<b>Date of Birth:</b> {}'.format(patient_info['Date of Birth']), styles['Normal']))
        story.append(Paragraph('<b>Gender:</b> {}'.format(patient_info['Gender']), styles['Normal']))
        story.append(Paragraph('<b>Ethnicity:</b> {}'.format(patient_info['Ethnicity']), styles['Normal']))
        story.append(Paragraph('<b>Family History:</b> {}'.format(patient_info['Family History']), styles['Normal']))
        story.append(Paragraph(f"Chromosome: {variant['chromosome']}", styles['Normal']))
        story.append(Paragraph(f"Position: {variant['position']}", styles['Normal']))
        story.append(Paragraph(f"Reference Allele: {variant['ref_allele']}", styles['Normal']))
        story.append(Paragraph(f"Alternate Allele: {variant['alt_allele']}", styles['Normal']))
        story.append(Paragraph(f"Quality: {variant['quality']}", styles['Normal']))
        story.append(Paragraph(f"Filter: {', '.join(variant['filter'])}", styles['Normal']))
        
        
        classification = classify_variant(variant)
        story.append(Paragraph(f"ACMG Classification: {classification}", styles['Normal']))
        
        story.append(Spacer(1, 12))
    
    doc.build(story)



vcf_file = 'normal_sample.deepvariant.vcf'  # Path to the VCF file
parsed_variants = parse_vcf(vcf_file)


patient_info = {
    'Patient ID': '[Insert Patient ID]',
    'Date of Birth': '[Insert Date of Birth]',
    'Gender': '[Insert Gender]',
    'Ethnicity': '[Insert Ethnicity]',
    'Family History': '[Insert relevant family history information]'
}

# Classify each variant
for variant in parsed_variants:
    variant['classification'] = classify_variant(variant)

# Generate the report
pdf_filename = 'Rayca_clinical_report-Yijia Sun.pdf'
generate_pdf_report(parsed_variants, pdf_filename)
